In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
from fake_useragent import UserAgent
from datetime import datetime
import time


In [2]:
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [3]:
local_time = datetime.now().strftime('%Y-%m-%d')

smtv_sports_url = "https://smtv.io/program/listwithgame/default?parent=43&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smtv_entertainment_url = "https://smtv.io/program/listwithgame/default?parent=44&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smtv_game_url = "https://smtv.io/program/listwithgame/default?parent=81&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smzb_sports_url = "https://smzb.io/program/listwithgame/default?parent=1&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smzb_entertainment_url = "https://smzb.io/program/listwithgame/default?parent=2&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
smzb_game_url = "https://smzb.io/program/listwithgame/default?parent=45&child=all&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
kl99_sports_url = "https://kl99.tv/program/listwithgame/default?parent=1&child=all&tag=&day_length=7&date="+local_time+"&now=Y&filter_live=Y"
lj88_sports_url = "https://lj88.tv/program/listwithgame/default?parent=27&child=all&tag=&day_length=7&date="+local_time+"&now=Y&filter_live=Y&from=gg"
ggzb_game_url = "https://ggzb.io/program/listwithgame/default?parent=1&child=all&tag=&day_length=7&date="+local_time+"&now=Y&filter_live=Y&from=gg"



In [4]:
# 網址

# smtv 體育
# https://smtv.io/program/43
# smtv 娛樂
# https://smtv.io/program/44
# smtv 電競
# https://smtv.io/program/81

# ------------------------

# smzb 體育
# https://smzb.io/program/1
# smzb 娛樂
# https://smzb.io/program/2
# smzb 電競    
# https://smzb.io/program/45

# ------------------------

# ggzb 電競
# https://ggzb.io/

# ------------------------

# lj88 體育
# https://lj88.tv/

# -----------------------------------------------------------------------


# 比較列表

# smtv體育 diff smzb體育
# smtv體育 diff kl99體育
# smtv體育 diff lj88體育

# ------------------------

# smtv電競 diff smzb電競
# smtv電競 diff ggzb電競

# ------------------------

# smtv娛樂 diff smzb娛樂


In [5]:
def Get_Schedule_to_DF(url):
    res = requests.get(url,headers=headers)
    schedule = json.loads(res.text)
    result = []
    for i in range(len(schedule["result"])):

        for j in range(len(schedule["result"][i]["data"])):

            live_types_name = schedule["result"][i]["data"][j]["live_types_name"]
            live_types_child_name = schedule["result"][i]["data"][j]["live_types_child_name"]
            lname = schedule["result"][i]["data"][j]["lname"]
            time = schedule["result"][i]["data"][j]["time"]
            hname = schedule["result"][i]["data"][j]["hname"]
            aname = schedule["result"][i]["data"][j]["aname"]
            account = []

            for k in schedule["result"][i]["data"][j]["account"]:
                nickname = k["nickname"]
                live_status = k["live_status"]

                if live_status == 1:
                    status = "開播"
                else:
                    status = "未開播"
                account.append({"nickname":nickname,"live_status":status})

            dict_={
                "live_types_name":live_types_name,
                "live_types_child_name":live_types_child_name,
                "lname":lname,
                "time":time,
                "hname":hname,
                "aname":aname,
                "account":account
            }
            result.append(dict_)

    df = pd.DataFrame(result)

    return df


In [6]:
# 含信號源
def Get_Schedule_diff_signal(url):
    res = requests.get(url,headers=headers)
    schedule = json.loads(res.text)
    result = []
    for i in range(len(schedule["result"])):

        for j in range(len(schedule["result"][i]["data"])):

            live_types_name = schedule["result"][i]["data"][j]["live_types_name"]
            live_types_child_name = schedule["result"][i]["data"][j]["live_types_child_name"]
            lname = schedule["result"][i]["data"][j]["lname"]
            time = schedule["result"][i]["data"][j]["time"]
            hname = schedule["result"][i]["data"][j]["hname"]
            aname = schedule["result"][i]["data"][j]["aname"]
            account = []

            for k in schedule["result"][i]["data"][j]["account"]:
                nickname = k["nickname"]
                live_status = k["live_status"]

                if live_status == 1:
                    status = "開播"
                else:
                    status = "未開播"
                account.append({"nickname":nickname,"live_status":status})

            result_str = time + "  " + lname + "  " + hname + " vs " + aname + "  " + str(account)


            result.append(result_str)

    return result

In [7]:
# 不含信號源
def Get_Schedule_diff(url):
    res = requests.get(url,headers=headers)
    schedule = json.loads(res.text)
    result = []
    for i in range(len(schedule["result"])):

        for j in range(len(schedule["result"][i]["data"])):

            live_types_name = schedule["result"][i]["data"][j]["live_types_name"]
            live_types_child_name = schedule["result"][i]["data"][j]["live_types_child_name"]
            lname = schedule["result"][i]["data"][j]["lname"]
            time = schedule["result"][i]["data"][j]["time"]
            hname = schedule["result"][i]["data"][j]["hname"]
            aname = schedule["result"][i]["data"][j]["aname"]

            result_str = time + "  " + lname + "  " + hname + " vs " + aname

            result.append(result_str)

    return result


In [8]:
def Schedule_diff(diff_A,diff_B):
    diff_1 = set(diff_A)
    diff_2 = set(diff_B)
    diff_result = sorted(list(diff_1.difference(diff_2)))
    return diff_result

In [11]:
# 比較賽事差異

print("----------以下為體育賽事----------")

smtv_sports = Get_Schedule_diff(smtv_sports_url)
smzb_sports = Get_Schedule_diff(smzb_sports_url)
diff_result = Schedule_diff(smtv_sports,smzb_sports)
print("分類:體育，smtv diff smzb")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    
# print("==============================================")

# smtv_sports = Get_Schedule_diff(smtv_sports_url)
# kl99_sports = Get_Schedule_diff(kl99_sports_url)
# diff_result = Schedule_diff(smtv_sports,kl99_sports)
# print("分類:體育，smtv diff kl99")
# print("數量:",len(diff_result))
# for i in diff_result:
#     print(i)
    
print("==============================================")

smtv_sports = Get_Schedule_diff(smtv_sports_url)
lj88_sports = Get_Schedule_diff(lj88_sports_url)
diff_result = Schedule_diff(smtv_sports,lj88_sports)
print("分類:體育，smtv diff lj88")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    
print("----------以下為娛樂直播----------")

smtv_entertainment = Get_Schedule_diff(smtv_entertainment_url)
smzb_entertainment = Get_Schedule_diff(smzb_entertainment_url)
diff_result = Schedule_diff(smtv_entertainment,smzb_entertainment)
print("分類:娛樂，smtv diff smzb")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    
print("----------以下為電競賽事----------")

smtv_game = Get_Schedule_diff(smtv_game_url)
smzb_game = Get_Schedule_diff(smzb_game_url)
diff_result = Schedule_diff(smtv_game,smzb_game)
print("分類:電競，smtv diff smzb")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    
print("==============================================")

smtv_game = Get_Schedule_diff(smtv_game_url)
ggzb_game = Get_Schedule_diff(ggzb_game_url)
diff_result = Schedule_diff(smtv_game,ggzb_game)
print("分類:電競，smtv diff ggzb")
print("數量:",len(diff_result))
for i in diff_result:
    print(i)
    




----------以下為體育賽事----------
分類:體育，smtv diff smzb
數量: 0
分類:體育，smtv diff lj88
數量: 23
2020-06-17 12:00:00  虚拟赛事  越南 vs 澳洲
2020-06-17 14:00:00  虚拟赛事  菲律宾 vs 土库曼
2020-06-17 16:00:00  虚拟赛事  苏联 vs 波兰
2020-06-17 20:30:00  国际友谊  皮耳森B队 vs 波希米亚人1905B队
2020-06-17 22:00:00  瑞典甲  耶夫勒 vs 托尔恩格鲁本
2020-06-17 23:00:00  国际友谊  布拉格斯拉维亚B队 vs 塔波斯科
2020-06-18 00:00:00  斯洛伐克杯  斯特里达 vs 鲁森比洛克
2020-06-18 00:00:00  瑞典甲  贝尔加 vs 哈宁厄
2020-06-18 00:30:00  国际友谊  皮斯克 vs 贝内绍夫
2020-06-18 01:00:00  匈甲  迪欧斯捷尔 vs 普斯卡斯学院
2020-06-18 01:00:00  瑞典甲  兰斯科罗纳 vs 艾斯基斯迈
2020-06-18 01:00:00  瑞典甲  卡尔斯塔德 vs 索伦蒂纳联
2020-06-18 01:00:00  瑞典甲  尼科平 vs 奥雷布洛西里安斯卡
2020-06-18 01:00:00  瑞典甲  布洛马波卡纳 vs 桑德维根斯
2020-06-18 01:00:00  瑞典甲  慕达拉 vs 斯柯维德
2020-06-18 01:00:00  瑞典甲  林多米 vs 隆德斯
2020-06-18 01:00:00  瑞典甲  柯维丁 vs 宁高平城
2020-06-18 01:00:00  瑞典甲  泰比 vs 希尔维亚
2020-06-18 01:00:00  瑞典甲  特罗尔海坦 vs IFK瓦纳默
2020-06-18 01:00:00  瑞典甲  阿西特拉保 vs 欧卡哈姆斯
2020-06-18 01:15:00  国际友谊  乌普萨拉 vs 凡纳
2020-06-18 01:30:00  瑞典甲  卡尔期克朗 vs 托纳斯
2020-06-18 01:30:00  瑞典甲  卡斯隆德 vs 费里塔比